## <font color=blue> Target (ground truth): NC county deaths

Need ...
To figure out what the "naive" answer is. It would be the county ratio multiplied by the state prediction. First, do this for 2021.




In [101]:
import pandas as pd

df = pd.read_csv('raw_data/nyt_us-counties-2021.csv')
# display(df.head(), len(df))

df.columns = map(lambda x: str(x).capitalize(), df.columns)

df = df[df.State=='North Carolina']
df = df.drop(['Cases','State'],axis=1)
df

# dff.to_csv('processed_data/nyt_nc_counties.csv',index=False)

,Date,County,Fips,Deaths
1903,2021-01-01,Alamance,37001.0,117.0
1904,2021-01-01,Alexander,37003.0,30.0
1905,2021-01-01,Alleghany,37005.0,2.0
1906,2021-01-01,Anson,37007.0,26.0
1907,2021-01-01,Ashe,37009.0,27.0
...,...,...,...,...
1184124,2021-12-31,Wayne,37191.0,332.0
1184125,2021-12-31,Wilkes,37193.0,205.0
1184126,2021-12-31,Wilson,37195.0,243.0
1184127,2021-12-31,Yadkin,37197.0,90.0


<font color=red> ^ NOTE: There is one extra row. Should be 36500 (365 * num counties) but it's 36501.

### Remove any samples with "Unknown" county.

In [102]:
df = df[df.County!="Unknown"]

### Format dataframe.

In [103]:
df = df.rename({'Deaths':'Target'},axis=1)
df['Target'] = df['Target'].astype('Int64')
df['Fips'] = df['Fips'].astype('Int64').astype('str')

df

,Date,County,Fips,Target
1903,2021-01-01,Alamance,37001,117
1904,2021-01-01,Alexander,37003,30
1905,2021-01-01,Alleghany,37005,2
1906,2021-01-01,Anson,37007,26
1907,2021-01-01,Ashe,37009,27
...,...,...,...,...
1184124,2021-12-31,Wayne,37191,332
1184125,2021-12-31,Wilkes,37193,205
1184126,2021-12-31,Wilson,37195,243
1184127,2021-12-31,Yadkin,37197,90


## <font color=blue> Prediction: Naive deaths per county.

### Get population of each county from covariates json. Just use whatever ones. 2019?

In [90]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)
print(len(d))

county_covs = []
for entry in d:
    if entry['state']=='NC':
        county_covs.append(entry)
print(len(county_covs))

3142
100


In [132]:
df['Pop'] = 0

df_pop = pd.DataFrame(columns=['Fips','Pop'])

for idx in range(len(county_covs)):
    pop = nc_covs[idx]['population']['2019']
    fips = nc_covs[idx]['fips']
    df_pop.loc[idx] = pd.Series({'Fips':fips, 'Pop':pop})
    
    
tot_pop = sum(df_pop.Pop)
tot_pop

df_pop['Ratio'] = df_pop['Pop'].div(tot_pop)
df_pop

,Fips,Pop,Ratio
0,37037,74470,0.0071
1,37001,169509,0.016162
2,37057,167609,0.015981
3,37069,69685,0.006644
4,37155,130625,0.012455
...,...,...,...
95,37175,34385,0.003278
96,37171,71783,0.006844
97,37043,11231,0.001071
98,37055,37009,0.003529


### Get state predictions for each week (?) or whatever.

In [ ]:
# # Need to read all files ...
# # NC FIPS is 37 !!!

# # # # # #
# state_fips = '37'
# # # # # #

# import os

# ddir = 'raw_data/covidhub-trained_ensemble-2021/'

# df_list = []
# for fname in os.listdir(ddir):
#     if fname.endswith('.csv'):
#         fpath = os.path.join(ddir, fname)
#         df = pd.read_csv(fpath)
#         df = df[df.location==state_fips]
#         df = df[df.target.isin(['1 wk ahead cum death',
#                         '2 wk ahead cum death',
#                         '3 wk ahead cum death',
#                         '4 wk ahead cum death'])]
#         df_list.append(df)
# #         print(fname, len(df))

# bigdf = pd.concat(df_list)
# bigdf.to_csv('processed_data/nc_state_projections.csv',index=False)
# bigdf

In [134]:
dff = pd.read_csv('processed_data/nc_state_projections.csv')
dff

,forecast_date,target,target_end_date,location,type,quantile,value
0,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.010,9620
1,2021-02-01,2 wk ahead cum death,2021-02-13,37,quantile,0.010,10034
2,2021-02-01,3 wk ahead cum death,2021-02-20,37,quantile,0.010,10391
3,2021-02-01,4 wk ahead cum death,2021-02-27,37,quantile,0.010,10712
4,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.025,9654
...,...,...,...,...,...,...,...
3835,2021-12-27,4 wk ahead cum death,2022-01-22,37,quantile,0.990,20660
3836,2021-12-27,1 wk ahead cum death,2022-01-01,37,point,NaN,19372
3837,2021-12-27,2 wk ahead cum death,2022-01-08,37,point,NaN,19568
3838,2021-12-27,3 wk ahead cum death,2022-01-15,37,point,NaN,19828


In [92]:
df

,Date,County,Fips,Target
1903,2021-01-01,Alamance,37001.0,117
1904,2021-01-01,Alexander,37003.0,30
1905,2021-01-01,Alleghany,37005.0,2
1906,2021-01-01,Anson,37007.0,26
1907,2021-01-01,Ashe,37009.0,27
...,...,...,...,...
1184124,2021-12-31,Wayne,37191.0,332
1184125,2021-12-31,Wilkes,37193.0,205
1184126,2021-12-31,Wilson,37195.0,243
1184127,2021-12-31,Yadkin,37197.0,90


In [ ]:
x

In [80]:
for k,v in nc_covs[2].items():
#     print(k,type(v))
    if isinstance(v,dict):
        print(k,v)
        print('')
    elif isinstance(v,list):
        print(k,v)
        print('')

noaa {'prcp': 44.3, 'snow': 4.5, 'temp': 59.2, 'altitude': 240.2, 'temp-jan': 39.3, 'temp-apr': 58.9, 'temp-jul': 78.1, 'temp-oct': 59.6}

zip-codes ['27360', '27012', '27374', '27284', '27262', '28127', '27107', '27351', '27239', '27294', '27373', '27299', '27127', '27265', '27292', '27295', '27293']

race {'non_hispanic_white_alone_male': 0.38832043625342316, 'non_hispanic_white_alone_female': 0.4054973181631058, 'black_alone_male': 0.047885256758288634, 'black_alone_female': 0.05294465094356508, 'asian_alone_male': 0.00722514900751153, 'asian_alone_female': 0.00898519769224803, 'hispanic_male': 0.03853015052890955, 'hispanic_female': 0.03574987023369867}

age {'0-4': 0.05488965389686711, '5-9': 0.05782505712700392, '10-14': 0.0646504662637448, '15-19': 0.060897684491882896, '20-24': 0.05463310442756654, '25-29': 0.061852287168350145, '30-34': 0.056476680846493925, '35-39': 0.05676902791616202, '40-44': 0.057538676324063745, '45-49': 0.07011556658651982, '50-54': 0.07326575541886175,

In [72]:
to_drop = []
for k,v in nc_covs[0].items():
    if isinstance(v,dict) or isinstance(v,list):
        to_drop.append(k)

# nc_covs_filtered = {k:v for k,v in nc_covs[0].items() if k not in to_drop}
# (nc_covs_filtered)

In [73]:
nc_covs_filtered = []

for cov in nc_covs:
    cov_filtered = {k:v for k,v in cov.items() if k not in to_drop}
    nc_covs_filtered.append(cov_filtered)
    
import csv

keys = nc_covs_filtered[0].keys()
with open('processed_data/TEMP_nc_county_covs.csv', 'w', newline='') as out:
    dict_writer = csv.DictWriter(out, keys)
    dict_writer.writeheader()
    dict_writer.writerows(nc_covs_filtered)

In [75]:
nc_covs = pd.read_csv('processed_data/TEMP_nc_county_covs.csv')
nc_covs

,name,fips,state,land_area (km^2),area (km^2),longitude (deg),latitude (deg),male,female,life-expectancy,police_deaths,avg_income,poverty-rate
0,chatham county,37037,NC,1765.011149,1836.275224,-79.229948,35.693740,35617,38853,80.80,0,63041,8.7
1,alamance county,37001,NC,1096.654286,1124.679535,-79.393317,36.041595,80533,88976,77.66,0,39342,14.6
2,davidson county,37057,NC,1432.711780,1470.164008,-80.230027,35.788877,81991,85618,76.57,0,39087,15.2
3,franklin county,37069,NC,1273.631713,1280.935745,-78.291038,36.079236,34516,35169,77.34,0,36238,11.6
4,robeson county,37155,NC,2453.425443,2458.522925,-79.106014,34.646941,62923,67702,73.73,0,29438,31.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,transylvania county,37175,NC,980.105287,985.251001,-82.801974,35.200530,16563,17822,80.33,0,42070,13.1
96,surry county,37171,NC,1382.436644,1392.723155,-80.693310,36.402419,34955,36828,76.73,0,38522,16.0
97,clay county,37043,NC,556.804508,571.822262,-83.747167,35.058649,5459,5772,78.43,0,34057,14.0
98,dare county,37055,NC,992.522471,3993.071812,-75.707896,35.667060,18153,18856,79.27,0,56314,8.9


In [ ]:
# for k,v in nc_covs[0].items():
# #     print(k,type(v))
#     if isinstance(v,dict):
#         print(k,v)
#         print('')
#     elif isinstance(v,list):
#         print(k,v)
#         print('')

In [ ]:
to_drop = []
for k,v in nc_covs[0].items():
    if isinstance(v,dict) or isinstance(v,list):
        to_drop.append(k)

# nc_covs_filtered = {k:v for k,v in nc_covs[0].items() if k not in to_drop}
# (nc_covs_filtered)

## NC (state) projections

In [64]:
# # Need to read all files ...
# # NC FIPS is 37 !!!

# # # # # #
# state_fips = '37'
# # # # # #

# import os

# ddir = 'raw_data/covidhub-trained_ensemble-2021/'

# df_list = []
# for fname in os.listdir(ddir):
#     if fname.endswith('.csv'):
#         fpath = os.path.join(ddir, fname)
#         df = pd.read_csv(fpath)
#         df = df[df.location==state_fips]
#         df = df[df.target.isin(['1 wk ahead cum death',
#                         '2 wk ahead cum death',
#                         '3 wk ahead cum death',
#                         '4 wk ahead cum death'])]
#         df_list.append(df)
# #         print(fname, len(df))

# bigdf = pd.concat(df_list)
# bigdf.to_csv('processed_data/nc_state_projections.csv',index=False)
# bigdf

In [65]:
dff = pd.read_csv('processed_data/nc_state_projections.csv')
dff

,forecast_date,target,target_end_date,location,type,quantile,value
0,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.010,9620
1,2021-02-01,2 wk ahead cum death,2021-02-13,37,quantile,0.010,10034
2,2021-02-01,3 wk ahead cum death,2021-02-20,37,quantile,0.010,10391
3,2021-02-01,4 wk ahead cum death,2021-02-27,37,quantile,0.010,10712
4,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.025,9654
...,...,...,...,...,...,...,...
3835,2021-12-27,4 wk ahead cum death,2022-01-22,37,quantile,0.990,20660
3836,2021-12-27,1 wk ahead cum death,2022-01-01,37,point,NaN,19372
3837,2021-12-27,2 wk ahead cum death,2022-01-08,37,point,NaN,19568
3838,2021-12-27,3 wk ahead cum death,2022-01-15,37,point,NaN,19828


## <font color=red> ^ Need to figure out which set of 'values' are the correct ones ...

# NC county covariates

In [20]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)
print(len(d))

3142


In [21]:
nc_covs = []
for entry in d:
    if entry['state']=='NC':
        nc_covs.append(entry)
print(len(nc_covs))

100


In [24]:
nc_covs[0].keys()

dict_keys(['name', 'fips', 'state', 'land_area (km^2)', 'area (km^2)', 'longitude (deg)', 'latitude (deg)', 'noaa', 'zip-codes', 'race', 'age', 'male', 'female', 'population', 'deaths', 'bls', 'life-expectancy', 'fatal_police_shootings', 'police_deaths', 'avg_income', 'covid-deaths', 'covid-confirmed', 'covid-vaccination', 'elections', 'edu', 'poverty-rate', 'cost-of-living', 'industry', 'health'])

In [37]:
# for k,v in nc_covs[0].items():
# #     print(k,type(v))
#     if isinstance(v,dict):
#         print(k,v)
#         print('')
#     elif isinstance(v,list):
#         print(k,v)
#         print('')

In [40]:
to_drop = []
for k,v in nc_covs[0].items():
    if isinstance(v,dict) or isinstance(v,list):
        to_drop.append(k)

# nc_covs_filtered = {k:v for k,v in nc_covs[0].items() if k not in to_drop}
# (nc_covs_filtered)

In [49]:
nc_covs_filtered = []

for cov in nc_covs:
    cov_filtered = {k:v for k,v in cov.items() if k not in to_drop}
    nc_covs_filtered.append(cov_filtered)
    
import csv

keys = nc_covs_filtered[0].keys()
with open('processed_data/nc_county_covs.csv', 'w', newline='') as out:
    dict_writer = csv.DictWriter(out, keys)
    dict_writer.writeheader()
    dict_writer.writerows(nc_covs_filtered)

In [48]:
# nc_covs_filtered